# DATA CLEANING

This notebook processes the raw data, extracting only the necessary information for further analysis and modeling.

## Libraries

In [ ]:
import googlemaps 
import os 
import pandas as pd
import pyarrow 
import re

## Constants

In [ ]:
data_path = "/Users/Jovan/Desktop/MDA-Project/Data"

input_path = "/0_DataOriginal"
output_path = "/1_DataClean"

google_api_key = ""
gmaps = googlemaps.Client(key = google_api_key)

# "P003 - HARTSTILSTAND - DOOD - OVERLEDEN",
# "P011 - PIJN OP DE BORST",
# "P039 - CARDIAAL PROBLEEM (NIET PIJN OP DE BORST)"
cardiac_codes = ["P003", "P011", "P039"]

belgium_north = 52
belgium_south = 49
belgium_west = 2.5
belgium_east = 6.4

## Functions

In [ ]:
def map_province_to_region(province):
    if province in ["Antwerp", "East Flanders", "Flemish Brabant", "Limburg", "West Flanders"]:
        return "Flanders"
    elif province in ["Hainaut", "Liege", "Luxembourg", "Namur", "Walloon Brabant"]:
        return "Wallonia"
    else:
        return "Brussels"

def insert_decimal(number, k):
    if number is None:
        return None
    else:
        num_str = str(number).replace('.', '')
        num_str = num_str[:k] + '.' + num_str[k:]
        return float(num_str)

## Importing data

In [ ]:
os.chdir(data_path + input_path)

for filename in os.listdir():
    if filename.endswith(".parquet.gzip"):
        df_name = filename.split('.')[0]
        globals()[df_name] = pd.read_parquet(filename)

## Cleaning

### AED locations

In [ ]:
# type

aed_locations['type'] = aed_locations['type'].str.lower()
aed_locations['type'] = aed_locations['type'].fillna("unknown")
aed_locations['type'] = aed_locations['type'].apply(lambda x: 
    'fix' if 'fixe' in x or 'vast' in x else (
        'mobile' if 'mobile' in x or 'm5066a' in x else 'unknown'
    )
)
aed_locations['type'] = pd.Categorical(aed_locations['type'], categories=["fix", "mobile", "unknown"])

In [ ]:
# public

aed_locations['public'] = aed_locations['public'].fillna("0")
aed_locations['public'] = aed_locations['public'].apply(lambda x: 
    1 if x.lower().startswith(('o', 'j', 'y')) else 0
)

In [ ]:
# geocoding

addresses = aed_locations['address'] + ", " + aed_locations['number'] + ", " + \
                 aed_locations['postal_code'] + ", " + aed_locations['municipality'] + ", " + \
                 aed_locations['province']

num = len(addresses)
confirmation = input(f"This will initialize {num} API requests. Are you sure? (yes/no): ")
if confirmation == "yes":
    print("OK. Geocoding...")
    geocoded = addresses.apply(lambda x: gmaps.geocode(x))
    
    latitude = geocoded.apply(lambda x: x[0]['geometry']['location']['lat'] if x else None)
    longitude = geocoded.apply(lambda x: x[0]['geometry']['location']['lng'] if x else None)
    coordinates = pd.DataFrame({'latitude': latitude, 'longitude': longitude})

    aed_locations = pd.concat([aed_locations, coordinates], axis=1)
else:
    print("OK. Will not procced.")

In [ ]:
# other

province_mapping = {
    "Hainaut": "Hainaut",
    "Liège": "Liege",
    "Bruxelles-Brussel": "Brussels",
    "Antwerpen": "Antwerp",
    "Vlaams-Brabant": "Flemish Brabant",
    "Luxembourg": "Luxembourg",
    "Limburg": "Limburg",
    "West-Vlaanderen": "West Flanders",
    "Oost-Vlaanderen": "East Flanders",
    "Brabant Wallon": "Walloon Brabant",
    "Namur": "Namur"
}

aed_locations['province'] = aed_locations['province'].map(province_mapping)
aed_locations['province'] = pd.Categorical(aed_locations['province'])

aed_locations['region'] = aed_locations['province'].apply(map_province_to_region)
aed_locations['region'] = pd.Categorical(aed_locations['region'])

aed_locations.rename(columns={'municipality': 'city'}, inplace=True)

In [ ]:
aeds = aed_locations[['type', 'public', 'latitude', 'longitude', 'city', 'province', 'region']]

### Interventions (Cards)

In [ ]:
interventions = pd.concat([interventions1, interventions2, interventions3], ignore_index=True)
del interventions1, interventions2, interventions3

In [ ]:
# Filtering only observations on cardiac events

cardiac_codes_string = '|'.join(cardiac_codes)

interventions = interventions[
    interventions['EventType Firstcall'].str.contains(cardiac_codes_string) |
    interventions['EventType Trip'].str.contains(cardiac_codes_string)
]

cad9['EventType Trip'] = cad9['EventType Trip'].fillna("unknown")
cad9 = cad9[cad9['EventType Trip'].str.contains(cardiac_codes_string)]

interventions_bxl = interventions_bxl[
    interventions_bxl['eventtype_firstcall'].str.contains(cardiac_codes_string) |
    interventions_bxl['eventtype_trip'].str.contains(cardiac_codes_string)
]

interventions_bxl2['EventType and EventLevel'] = interventions_bxl2['EventType and EventLevel'].fillna("unknown")
interventions_bxl2 = interventions_bxl2[
    interventions_bxl2['EventType and EventLevel'].str.contains(cardiac_codes_string)
]

In [ ]:
# Selecting only columns are actually needed

selected_columns = ["Mission ID", "Latitude intervention", "Longitude intervention", 
                    "CityName intervention", "EventType Trip", "Vector type"]
interventions = interventions[selected_columns]

cad9.rename(columns={'Vector Type': 'Vector type'}, inplace=True)

cad9 = cad9[selected_columns]

selected_columns = ["mission_id", "latitude_intervention", "longitude_intervention",
                    "cityname_intervention", "eventtype_trip", "vector_type"]
interventions_bxl = interventions_bxl[selected_columns]

selected_columns = ["Mission ID", "Latitude intervention", "Longitude intervention",
                    "Cityname Intervention", "EventType and EventLevel", "Vector type NL"]
interventions_bxl2 = interventions_bxl2[selected_columns]

In [ ]:
# Naming them consistently, so we can merge them into one dataset

colnames = ["id", "latitude", "longitude", "city", "event", "vehicle"]

interventions.columns = colnames
cad9.columns = colnames
interventions_bxl.columns = colnames
interventions_bxl2.columns = colnames

cards = pd.concat([interventions, cad9, interventions_bxl, interventions_bxl2], ignore_index=True)

del interventions, cad9, interventions_bxl, interventions_bxl2, colnames, selected_columns

In [ ]:
# Vehicle

cards['vehicle'] = cards['vehicle'].astype(str).apply(lambda x: 
    'ambulance' if re.search(r'AMB|Ambulance|Ambulance Event|Ambulance Exceptional|Brandziekenwagen|Decontanimatieziekenwagen', x) 
    else ('pit' if re.search(r'PIT', x) 
    else ('mug' if re.search(r'MUG|MUG Event', x) 
    else None))
)
cards['vehicle'] = pd.Categorical(cards['vehicle'], categories=["ambulance", "pit", "mug"], ordered=True)

In [ ]:
# Event

cards['event'] = cards['event'].astype(str).apply(lambda x: 
    3 if re.search(cardiac_codes[0], str(x)) 
    else (11 if re.search(cardiac_codes[1], str(x)) 
    else (39 if re.search(cardiac_codes[2], str(x)) 
    else None))
)
cards['event'] = pd.Categorical(cards['event'], categories=[3, 11, 39], ordered=True)

In [ ]:
# Latitude, longitude

# 2 - Correct ones
cards2 = cards[
    (cards['latitude'] >= belgium_south) & (cards['latitude'] <= belgium_north) &
    (cards['longitude'] >= belgium_west) & (cards['longitude'] <= belgium_east)
]

# 3 - Ones that can be fixed (no NAs)
cards3 = cards[
    (cards['latitude'] < belgium_south) | (cards['latitude'] > belgium_north) |
    (cards['longitude'] < belgium_west) | (cards['longitude'] > belgium_east)
]
cards3 = cards3[~cards3['latitude'].isna() & ~cards3['longitude'].isna()]

# Fixing the wrong ones
cards3['latitude'] = cards3['latitude'].apply(lambda x: x / 10 if 100 <= x < 1000 else x)
cards3['latitude'] = cards3['latitude'].apply(lambda x: insert_decimal(x, 2) if x >= 1000 else x)

cards3['longitude'] = cards3['longitude'].apply(lambda x: x / 10 if 10 <= x < 100 else (x / 100 if 100 <= x < 1000 else x))
cards3['longitude'] = cards3['longitude'].apply(lambda x: insert_decimal(x, 1) if x >= 1000 else x)

# Concatenate corrected and correctly located ones
cards = pd.concat([cards2, cards3])

# Filter outlying values
cards = cards[
    (cards['latitude'] >= belgium_south) & (cards['latitude'] <= belgium_north) &
    (cards['longitude'] >= belgium_west) & (cards['longitude'] <= belgium_east)
]

## Exporting

In [ ]:
os.chdir(output_folder_path + output_path)

aeds.to_csv("aeds.csv", index=False)
cards.to_csv("cards.csv", index=False)